In [1]:
import cv2
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

In [3]:
masks = pd.read_csv(os.path.join('data', 'train_ship_segmentations.csv'))
masks.head()

,ImageId,EncodedPixels
0,00003e153.jpg,NaN
1,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
2,00021ddc3.jpg,101361 1 102128 3 102896 4 103663 6 104430 9 1...
3,00021ddc3.jpg,95225 2 95992 5 96760 7 97527 9 98294 9 99062 ...
4,00021ddc3.jpg,74444 4 75212 4 75980 4 76748 4 77517 3 78285 ...


In [9]:
for idx, row in tqdm(masks.iterrows()):
    if row['ImageId'] == 'dba4b574c.jpg':
        if str(row['EncodedPixels']) != 'nan':
            mask_arr = rle_decode(str(row['EncodedPixels']))
        else:
            mask_arr = np.zeros(shape=(768, 768), dtype=np.uint8)
        cv2.imwrite(os.path.join('data', row['ImageId']), mask_arr)

131030it [00:04, 27850.50it/s]


In [13]:
np.where(mask_arr == 1)

(array([364, 364, 365, 365, 365, 365, 365, 366, 366, 366, 366, 366, 366,
        366, 366, 366, 367, 367, 367, 367, 367, 367, 367, 367, 367, 367,
        367, 368, 368, 368, 368, 368, 368, 368, 368, 368, 368, 368, 368,
        369, 369, 369, 369, 369, 369, 369, 369, 369, 370, 370, 370, 370,
        370, 370, 371, 371]),
 array([600, 601, 597, 598, 599, 600, 601, 594, 595, 596, 597, 598, 599,
        600, 601, 602, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601,
        602, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603,
        593, 594, 595, 596, 597, 598, 599, 600, 601, 593, 594, 595, 596,
        597, 598, 594, 595]))

In [3]:
from fastai.conv_learner import *
from fastai.dataset import *
from fastai.models.resnet import vgg_resnet50
import glob

In [4]:
torch.cuda.set_device(0)

In [5]:
torch.backends.cudnn.benchmark=True

In [6]:
torch.cuda.is_available()

True

In [7]:
PATH = Path('.')
TRAIN_DN = 'data/train'
TRAIN_LABEL_DN = 'data/train_mask'

In [8]:
class MatchedFilesDataset(FilesDataset):
    def __init__(self, fnames, y, transform, path):
        self.y=y
        assert(len(fnames)==len(y))
        super().__init__(fnames, transform, path)
    def get_y(self, i): return open_image(os.path.join(self.path, self.y[i]))
    def get_c(self): return 0

In [9]:
x_names = np.array(glob.glob(str(Path(TRAIN_DN)/f'*.jpg')))
y_names = np.array(glob.glob(str(Path(TRAIN_LABEL_DN)/f'*.jpg')))

In [10]:
aug_tfms = [RandomRotate(4, tfm_y=TfmType.CLASS),
            RandomFlip(tfm_y=TfmType.CLASS),
            RandomLighting(0.05, 0.05, tfm_y=TfmType.CLASS)]

In [11]:
len(x_names), len(y_names)

(104070, 104070)

In [12]:
val_idxs = list(range(10407))
((val_x,trn_x),(val_y,trn_y)) = split_by_idx(val_idxs, x_names, y_names)

In [13]:
sz = 768
bs = 7

In [14]:
tfms = tfms_from_model(resnet34, sz, crop_type=CropType.NO, tfm_y=TfmType.CLASS, aug_tfms=aug_tfms)

In [15]:
datasets = ImageData.get_ds(MatchedFilesDataset, (trn_x,trn_y), (val_x,val_y), tfms, path=PATH)

In [16]:
md = ImageData(PATH, datasets, bs, num_workers=8, classes=None)
denorm = md.trn_ds.denorm

In [17]:
x,y = next(iter(md.trn_dl))

In [18]:
x.shape, y.shape

(torch.Size([6, 3, 768, 768]), torch.Size([6, 768, 768]))

In [19]:
f = resnet34
cut,lr_cut = model_meta[f]

In [20]:
def get_base():
    layers = cut_model(f(True), cut)
    return nn.Sequential(*layers)

In [21]:
def dice(pred, targs):
    pred = (pred>0).float()
    return 2. * ((pred*targs).sum()+1e-8)/ ((pred+targs).sum()+1e-8)

In [22]:
class StdUpsample(nn.Module):
    def __init__(self, nin, nout):
        super().__init__()
        self.conv = nn.ConvTranspose2d(nin, nout, 2, stride=2)
        self.bn = nn.BatchNorm2d(nout)
        
    def forward(self, x): return self.bn(F.relu(self.conv(x)))

In [23]:
class Upsample34(nn.Module):
    def __init__(self, rn):
        super().__init__()
        self.rn = rn
        self.features = nn.Sequential(
            rn, nn.ReLU(),
            StdUpsample(512,256),
            StdUpsample(256,256),
            StdUpsample(256,256),
            StdUpsample(256,256),
            nn.ConvTranspose2d(256, 1, 2, stride=2))
        
    def forward(self,x): return self.features(x)[:,0]

In [24]:
class UpsampleModel():
    def __init__(self,model,name='upsample'):
        self.model,self.name = model,name

    def get_layer_groups(self, precompute):
        lgs = list(split_by_idxs(children(self.model.rn), [lr_cut]))
        return lgs + [children(self.model.features)[1:]]

In [25]:
m_base = get_base()

In [26]:
m = to_gpu(Upsample34(m_base))
models = UpsampleModel(m)

In [27]:
learn = ConvLearner(md, models)
learn.opt_fn=optim.Adam
learn.crit=nn.BCEWithLogitsLoss()
learn.metrics=[accuracy_thresh(0.5), dice]

In [28]:
learn.freeze_to(1)

In [35]:
learn.lr_find()
learn.sched.plot()

TypeError: eq received an invalid combination of arguments - got (torch.cuda.FloatTensor), but expected one of:
 * (int value)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.cuda.FloatTensor[0m)
 * (torch.cuda.LongTensor other)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.cuda.FloatTensor[0m)


In [28]:
lr=1e-4
wd=1e-7
lrs = np.array([lr/100,lr/10,lr])/2

In [30]:
learn.fit(lr,1, wds=wd, cycle_len=4, use_clr=(20,8))

epoch      trn_loss   val_loss   <lambda>   dice             
    0      0.423869   0.364539   0.998844   0.0       
    1      0.123915   0.110569   0.999207   0.12             
    2      0.061287   0.058113   0.999201   0.0               
    3      0.045886   0.045344   0.999207   0.12              



[array([0.04534]), 0.999207239151001, 0.1200000137269676]

In [31]:
learn.save('tmp')

In [29]:
learn.load('tmp')

In [30]:
learn.unfreeze()
learn.bn_freeze(True)

In [31]:
learn.fit(lrs,1,cycle_len=4,use_clr=(20,8))

epoch      trn_loss   val_loss   <lambda>   dice                      
    0      3.3e-05    3e-05      0.998993   0.292303  
 24%|██▎       | 3677/15611 [27:59<1:30:52,  2.19it/s, loss=3.07e-05]

KeyboardInterrupt: 

In [32]:
learn.save('tmp')

In [65]:
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [50]:
TEST_DN = 'data/test'

In [66]:
for fn in tqdm(glob.glob(str(Path(TEST_DN)/f'*.jpg'))):
    img = open_image(fn)
    img = img.T
    pred = learn.model(V(img[np.newaxis,:,:,:]))
    pred_str = rle_encode(pred.cpu().data.numpy())
    break

  0%|          | 0/88500 [00:00<?, ?it/s]


In [67]:
pred.shape

torch.Size([1, 768, 768])

In [68]:
type(pred)

torch.autograd.variable.Variable

In [69]:
pred_str

'1 1 3 1 5 1 7 1 9 1 11 1 13 1 15 1 17 1 19 1 21 1 23 1 25 1 27 1 29 1 31 1 33 1 35 1 37 1 39 1 41 1 43 1 45 1 47 1 49 1 51 1 53 1 55 1 57 1 59 1 61 1 63 1 65 1 67 1 69 1 71 1 73 1 75 1 77 1 79 1 81 1 83 1 85 1 87 1 89 1 91 1 93 1 95 1 97 1 99 1 101 1 103 1 105 1 107 1 109 1 111 1 113 1 115 1 117 1 119 1 121 1 123 1 125 1 127 1 129 1 131 1 133 1 135 1 137 1 139 1 141 1 143 1 145 1 147 1 149 1 151 1 153 1 155 1 157 1 159 1 161 1 163 1 165 1 167 1 169 1 171 1 173 1 175 1 177 1 179 1 181 1 183 1 185 1 187 1 189 1 191 1 193 1 195 1 197 1 199 1 201 1 203 1 205 1 207 1 209 1 211 1 213 1 215 1 217 1 219 1 221 1 223 1 225 1 227 1 229 1 231 1 233 1 235 1 237 1 239 1 241 1 243 1 245 1 247 1 249 1 251 1 253 1 255 1 257 1 259 1 261 1 263 1 265 1 267 1 269 1 271 1 273 1 275 1 277 1 279 1 281 1 283 1 285 1 287 1 289 1 291 1 293 1 295 1 297 1 299 1 301 1 303 1 305 1 307 1 309 1 311 1 313 1 315 1 317 1 319 1 321 1 323 1 325 1 327 1 329 1 331 1 333 1 335 1 337 1 339 1 341 1 343 1 345 1 347 1 349 1 351 